In [1]:
# coding: utf-8
import ast
import os
import sys
import re
from pprint import pprint
import numpy as np
from tqdm import tqdm

from func import normalization as norm

##############################################
# JUPYTER-NOTEBOOK FOR NORMALIZING BARCODE   #
# COUNTS FROM BFG-PCA SCREENINGS             #
# Last modified by Daniel Evans-Yamamoto     #
##############################################

PATH = os.path.abspath(".")
run_name = "test"
norm_dir = '%s/Data/%s/normalization'%(PATH,run_name)
stats_dir = '%s/Data/%s/stats'%(PATH,run_name)
if not os.path.isdir(norm_dir):
    os.makedirs(norm_dir)
    os.makedirs(stats_dir)

In [2]:
#  Definfing the file location
count_f        = "%s/Data/%s/barcode_counts/counts.txt"%(PATH,run_name)

#  Reading dict object from the text files with count data
count     = norm.reading(count_f)
# Reading tag information and barcode database 
multiplex_tag  = norm.get_tagdata('%s/%s/%s_tag.csv'%(PATH,run_name,run_name))#Change this to your own file. Read wiki for instructions.
db             = norm.get_mapdata('%s/%s/%s_database.csv'%(PATH,run_name,run_name))#Change this to your own file. Read wiki for instructions.

In [3]:
# Examine total reads;
norm.tot_reads(count,multiplex_tag)

Index Sample Total reads
P01-P01 DMSO_72 1962
P02-P02 DMSO_96 1589
P03-P03 MTX1_72 2370
P04-P04 MTX1_96 1484
P05-P05 MTX10_72 4536
P06-P06 MTX10_96 3816
P07-P07 MTX100_72 6278
P08-P08 MTX100_96 5392
P09-P09 MTX200_72 5295
P10-P10 MTX200_96 5420


In [4]:
#pprint(count)

In [5]:
# Organize dict of reads per strain
count_2 =  norm.organize_data(count,db,multiplex_tag,"PCA")

Working on ...
P01-P01 Control_1
P03-P03 MTX1_72_PRS
P05-P05 MTX10_72_PRS
P07-P07 MTX100_72_PRS
P09-P09 MTX200_72_PRS
P02-P02 Control_2
P04-P04 MTX1_96_PRS
P06-P06 MTX10_96_PRS
P08-P08 MTX100_96_PRS
P10-P10 MTX200_96_PRS


In [6]:
# Eliminating strains present in less than 90% of diploids
count_3 = norm.absent(count_2,0.9,"PCA")

count_sums = norm.get_sums(count_3,multiplex_tag,"PCA",db)




In [7]:
# Normalize scores based on strain abundance of bait and prey haploids in Control condition

raw_s,hap_s  = norm.compute_s(count_3,count_sums,db,multiplex_tag,"PCA")

Control_1 		Bait: , (11 ORFs) 11 Prey, (9 ORFs) 9
Control_2 		Bait: , (10 ORFs) 10 Prey, (9 ORFs) 9
Generated : /Users/danyamamotoevans/GitHub/BFG/Data/test/normalization/Haploid_F.csv


In [9]:
# Organizing data and counting haploid strains
reps= norm.count_haploids(hap_s,norm_dir,db)
norm.haploid_replicates(reps,norm_dir)

Control_1 		Bait: , (11 ORFs) 11 Prey, (9 ORFs) 9
Control_2 		Bait: , (10 ORFs) 10 Prey, (9 ORFs) 9
Generated : /Users/danyamamotoevans/GitHub/BFG/Data/test/normalization/Haploid_F.csv
Generated : /Users/danyamamotoevans/GitHub/BFG/Data/test/normalization/haplliod_n.csv


In [10]:
## Normalize signal 's'
## For Y2H ; Normalize data by Nth quantile of bait 'median distribution' of 's' score
## For PCA ; Normalize data by median 's' score of both bait and prey 

normalized_score,hap_s = norm.compute_ds(raw_s,count_sums,db,multiplex_tag,"PCA")


In [12]:
norm.output_norm_score(normalized_score,db,norm_dir)

100%|██████████| 11/11 [00:00<00:00, 213.62it/s]

Generated : /Users/danyamamotoevans/GitHub/BFG/Data/test/normalization/BFG-PCA_scores_Normalized.csv


In [ ]:
output_stats(data,db,norm_dir)